In [1]:
!pip install -q open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.4 MB/s eta 0:00:00


In [2]:
import open3d as o3d
import numpy as np

# Task 1
In the given point cloud, there is a shoe kept on the floor. The task is to detect the
floor and re-orient the floor (and hence the entire point cloud) on the ~~YZ~~ XZ plane with
the centre of the floor lying on the origin i.e. the equation of the plane passing through
the floor should be y = 0.

In [3]:
def get_rotation_matrix(v1, v2):
  """Compute rotation matrix for orienting v1 to v2"""
  v1 = v1 / np.linalg.norm(v1)
  v2 = v2 / np.linalg.norm(v2)

  rot_axis = np.cross(v1, v2)
  rot_axis /= np.linalg.norm(rot_axis)

  rot_angle = np.arccos(np.dot(v1, v2))

  rot_mat = o3d.geometry.get_rotation_matrix_from_axis_angle(rot_angle * rot_axis)
  return rot_mat

def show_pcd(pcd):
  o3d.visualization.draw_plotly([pcd], up=[0, 1, 0])

def reorient_pcd(pcd, visualize=False):
  if visualize:
    print("Raw pcd:")
    show_pcd(pcd)

  # Detect floor using plane segmentation
  plane_eq, inliers = pcd.segment_plane(distance_threshold=0.001,
                                      ransac_n=5,
                                      num_iterations=1000)
  plane = pcd.select_by_index(inliers)

  if visualize:
    print("Segmented floor:")
    show_pcd(plane)

  # Translate to bring floor center to origin
  T = -plane.get_center()
  plane.translate(T)
  pcd.translate(T)

  # Get plane normal from eq
  [a, b, c, d] = plane_eq
  plane_normal = np.array([a, b, c], dtype=float)

  # Orient normal towards pcd center
  if np.dot(plane_normal, pcd.get_center()) < 0:
    plane_normal = -plane_normal

  # Compute rotation matrix
  y_axis = np.array([0, 1, 0], dtype=float)
  R = get_rotation_matrix(plane_normal, y_axis)

  # Rotate pcd to align normal with y axis
  plane.rotate(R, center=[0, 0, 0])
  pcd.rotate(R, center=[0, 0, 0])

  if visualize:
    print("Re-oriented pcd:")
    show_pcd(pcd)

  return pcd, R, T

In [4]:
pcd = o3d.io.read_point_cloud("/content/drive/MyDrive/Clipboard/pc_assignment/shoe_pc.ply")
pcd_aligned, R, T = reorient_pcd(pcd, visualize=False)

## Task 2
Since the 3D representation involved over here is a point cloud, the overall scene looks kind of pointy and perforated. Convert this point cloud to some other representation where the scene looks more continuous with a smooth surface.


In [5]:
def meshify_pcd(pcd, visualize=False):
  # Remove outliers
  pcd_clean, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.5)

  if visualize:
    print("Clean pcd:")
    show_pcd(pcd_clean)

  # Calculate normals and orient them for consistency
  pcd_clean.estimate_normals(
      search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
  pcd_clean.orient_normals_consistent_tangent_plane(k=30)

  # Construct mesh
  mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
      pcd_clean, depth=5)

  if visualize:
    print("Meshified:")
    show_pcd(mesh)

  return mesh

In [6]:
meshify_pcd(pcd_aligned, visualize=False)

TriangleMesh with 1623 points and 3108 triangles.

# Task 3
Add a unit test case to check and verify your solution. For this, add some random transformations to an input point cloud and then pass it through your algorithm to test it.

In [7]:
def unit_test(visualize=False):
  # Create point cloud on the XZ plane
  points = np.array([[x, 0, z] for x in np.arange(-1, 1.05, 0.05) for z in np.arange(-1, 1.05, 0.05)])
  points = np.concatenate((points, np.array([[0, 1, 0]])), axis=0) # hack for visualization
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(points)

  if visualize:
    print("Init plane:")
    show_pcd(pcd)

  # Apply random rotation and translation
  angle = np.random.uniform(0, np.pi)
  axis = np.random.randn(3)
  axis[1] = 0 # do not rotate about Y since algorithm allows any rotation about Y
  axis /= np.linalg.norm(axis)
  R = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)
  pcd.rotate(R, center=[0, 0, 0])
  T = np.random.uniform(-1, 1, size=3)
  pcd.translate(T)

  _, R_hat, T_hat = reorient_pcd(pcd, visualize=visualize)
  R_prod = np.matmul(R_hat, R)
  T_sum  = T_hat + T
  print(R_prod)
  print(T_sum)
  # R_hat * R = 1
  # T_hat + T = 0
  if np.allclose(R_prod, np.eye(3, dtype=float)) and np.allclose(T_sum, np.zeros_like(T)):
    print("Test passed.")
  else:
    print("Test failed")

unit_test(visualize=False)

[[ 1.00000000e+00 -3.97491755e-16 -7.45206385e-17]
 [ 3.63890903e-16  1.00000000e+00 -1.07006905e-16]
 [-4.17507078e-18  2.92120513e-17  1.00000000e+00]]
[ 1.38777878e-16  5.55111512e-16 -1.11022302e-15]
Test passed.
